In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
%env MODEL_PATH=../model/deepspeech-0.7.4-models.pbmm

env: MODEL_PATH=../model/deepspeech-0.7.4-models.pbmm


In [5]:
from src.deepspeech_stt import deepspeech_predict, SIGNAL_FILTERS
from src.scoring import english_scoring

import pandas as pd
import numpy as np

In [36]:
vf_samples = [
    ("../samples/test.wav", "test one two three test one two three"),
    ("samples/voxforge/vf5-11.wav", "when you come back in a few days bring eileen"),
    ("samples/voxforge/vf5-12.wav", "gregson has left the outer door slightly ajar"),
    ("samples/voxforge/vf5-13.wav", "the date was nearly thirteen years old"),
    ("samples/voxforge/vf5-14.wav", "they were the presage of storm"),
    ("samples/voxforge/vf5-15.wav", "down there the earth was already swelling with life")
]

In [37]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [45]:
y = []

grid = [
    [""],
    ["high_pass_filter"], 
    ["low_pass_filter"], 
    ["butter_bandpass_filter"], 
    ["logmmse_denoise"],
    ["high_pass_filter", "logmmse_denoise"], 
    ["low_pass_filter", "logmmse_denoise"], 
    ["butter_bandpass_filter", "logmmse_denoise"],
    ["logmmse_denoise", "high_pass_filter"], 
    ["logmmse_denoise", "low_pass_filter"], 
    ["logmmse_denoise", "butter_bandpass_filter"]  
]

for filters in grid:
    for s, gt in vf_samples:
        y_pred = deepspeech_predict(s, filters=filters, batch_after_silence=True)
        y.append((gt, y_pred[0], filters))

In [46]:
y

[('test one two three test one two three',
  'test one two three test one two three',
  ['']),
 ('when you come back in a few days bring eileen',
  'when you come back and a few days bring ileen',
  ['']),
 ('gregson has left the outer door slightly ajar',
  'greaks ten has left the outer door slagte me ajar',
  ['']),
 ('the date was nearly thirteen years old',
  'they de was nearly thirteen years old',
  ['']),
 ('they were the pressage of storm', 'they were the pressage of storm', ['']),
 ('down there the earth was already swelling with life',
  "i on there so you're wasalready swelling with life",
  ['']),
 ('test one two three test one two three',
  'tust one two three task one two three',
  ['high_pass_filter']),
 ('when you come back in a few days bring eileen',
  'when you come back and a few days bring ileen',
  ['high_pass_filter']),
 ('gregson has left the outer door slightly ajar',
  'greaks ten has left the outer door slaight in the ajar',
  ['high_pass_filter']),
 ('the d

In [47]:
# collect candidates from each filter
all_candidates = []
for i in range(len(vf_samples)):
    all_candidates.append(y[i::6])
    
all_candidates[0]

[('test one two three test one two three',
  'test one two three test one two three',
  ['']),
 ('test one two three test one two three',
  'tust one two three task one two three',
  ['high_pass_filter']),
 ('test one two three test one two three',
  'test one two three test one two three',
  ['low_pass_filter']),
 ('test one two three test one two three',
  'tust one to three test one to three',
  ['butter_bandpass_filter']),
 ('test one two three test one two three',
  'tust one two three task one two three',
  ['logmmse_denoise']),
 ('test one two three test one two three',
  'tus one two three test one to three',
  ['high_pass_filter', 'logmmse_denoise']),
 ('test one two three test one two three',
  'thust one two three test one to three',
  ['low_pass_filter', 'logmmse_denoise']),
 ('test one two three test one two three',
  'tus one to three test one to three',
  ['butter_bandpass_filter', 'logmmse_denoise']),
 ('test one two three test one two three',
  'tus one two three test 

In [48]:
most_en = []
best_jac = []
for item in all_candidates:
    choices = [i for _,i,_ in item]

    en_scores = english_scoring(choices)
    jac_scores = [jaccard(y, y_pred) for y, y_pred, _ in item]
    
    en_idx = en_scores.index(max(en_scores))
    jac_idx = jac_scores.index(max(jac_scores))

    most_en.append(item[en_idx] + (max(en_scores),))
    best_jac.append(item[jac_idx] + (max(jac_scores),))

In [49]:
df_en = pd.DataFrame(most_en, columns = ["Ground Truth", "Best Prediction", "Filter", "% English"])
df_en

,Ground Truth,Best Prediction,Filter,% English
0,test one two three test one two three,test one two three test one two three,[],1.000000
1,when you come back in a few days bring eileen,when you come back and a few days bring ileen,[],0.900000
2,gregson has left the outer door slightly ajar,greaks ten has left the outer door slate me a jar,[low_pass_filter],0.818182
3,the date was nearly thirteen years old,they de was nearly thirteen years old,[],0.857143
4,they were the pressage of storm,they were the pressage of storm,[],0.833333
5,down there the earth was already swelling with...,de on there so you're was already swelling wit...,[low_pass_filter],0.900000


In [50]:
df_jac = pd.DataFrame(best_jac, columns = ["Ground Truth", "Best Prediction", "Filter", "Jaccard Score"])
df_jac

,Ground Truth,Best Prediction,Filter,Jaccard Score
0,test one two three test one two three,test one two three test one two three,[],1.000000
1,when you come back in a few days bring eileen,when you come back in a few days bring ileen,"[low_pass_filter, logmmse_denoise]",0.818182
2,gregson has left the outer door slightly ajar,graeksnin has left the outer door slightly ajar,"[logmmse_denoise, butter_bandpass_filter]",0.777778
3,the date was nearly thirteen years old,the the was nearly thirteen years old,[high_pass_filter],0.857143
4,they were the pressage of storm,they were the pressage of storm,[],1.000000
5,down there the earth was already swelling with...,down there sears was already swelling with life,"[low_pass_filter, logmmse_denoise]",0.700000
